In [1]:
import tensorflow
from util import PATH_FOR_OUR_TRAINING_DATA, UPPER_LIMIT_OF_IMAGES, TARGET_SHAPE
import pandas as pd
from DS_Generator import DS_Generator, smart_batches
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_datasets as tfds
from util import PATH_FOR_OUR_TRAINING_DATA

In [2]:
df = pd.read_csv(PATH_FOR_OUR_TRAINING_DATA + "/data.csv", index_col=0)

In [3]:
train_ds, val_ds = DS_Generator().generate(df,0.1,2)

Since none Batch-size was specified we, took the 64 specified in utils.py
We threw away the datapoint with index 23785 


NotImplementedError: in user code:

    File "C:\Users\Felix\Desktop\HappyWhale\happywhale\DS_Generator.py", line 253, in augment1  *
        x = tf.convert_to_tensor(tf.keras.preprocessing.image.random_brightness(x, (0.4,0.8)))
    File "C:\Users\Felix\anaconda3\lib\site-packages\keras_preprocessing\image\affine_transformations.py", line 242, in random_brightness  *
        return apply_brightness_shift(x, u)
    File "C:\Users\Felix\anaconda3\lib\site-packages\keras_preprocessing\image\affine_transformations.py", line 215, in apply_brightness_shift  *
        x = array_to_img(x)
    File "C:\Users\Felix\anaconda3\lib\site-packages\keras_preprocessing\image\utils.py", line 254, in array_to_img  *
        x = np.asarray(x, dtype=dtype)
    File "C:\Users\Felix\anaconda3\lib\site-packages\numpy\core\_asarray.py", line 102, in asarray  **
        return array(a, dtype, copy=False, order=order)

    NotImplementedError: Cannot convert a symbolic Tensor (args_0:0) to a numpy array. This er

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=64, kernel_size=2, padding='same', activation='relu', input_shape=TARGET_SHAPE+(1,)),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation=None), # No activation on final dense layer
    tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1)) # L2 normalize embeddings

])

model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tfa.losses.TripletSemiHardLoss())


In [ ]:
model.fit(
    train_ds,
    epochs=1,
    validation_data=val_ds
)